# Analise de dados - Agressividade condutores de veiculos


## Coleta de Dados
- Com os dados salvos dentro do Fiware, é preciso ler cada atributo para cada entidade

In [ ]:
#Lê todos os dados que existem para uma entidade, e junta todos os atributos em um dataframe. Preciso modificar a entidade na mão

import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import json

    
def fetch_paginated_data(url_base, attribute_name, h_limit=100, is_location=False):
    """
    Busca dados usando paginação com hLimit e hOffset.
    """
    dict_data = {attribute_name: [], 'recvTime_' + attribute_name: []}
    h_offset = 0

    while True:
        # Construir URL com hLimit e hOffset
        url = f"{url_base}?hLimit={h_limit}&hOffset={h_offset}"
        headers = {
            'fiware-service': 'smart',
            'fiware-servicepath': '/'
        }
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            values = data['contextResponses'][0]['contextElement']['attributes'][0]['values']
            if not values:
                break  # Interrompe se não houver mais valores

            for value in values:
                dict_data[attribute_name].append(
                    value['attrValue']['coordinates'] if is_location else value['attrValue']
                )
                dict_data['recvTime_' + attribute_name].append(value['recvTime'])

            # Incrementa o offset para a próxima página
            h_offset += h_limit
        else:
            print(f"Erro na requisição: {response.status_code}")
            break

    return pd.DataFrame(dict_data)


# Definir atributos e URLs
attributes = [
    ("velocidade", "http://20.62.95.153:8666/STH/v1/contextEntities/type/iot/id/urn:ngsi-ld:eduardo:001/attributes/velocidade"),
    ("rpm", "http://20.62.95.153:8666/STH/v1/contextEntities/type/iot/id/urn:ngsi-ld:eduardo:001/attributes/rpm"),
    ("temperatura", "http://20.62.95.153:8666/STH/v1/contextEntities/type/iot/id/urn:ngsi-ld:eduardo:001/attributes/temperature"),
    ("pressao", "http://20.62.95.153:8666/STH/v1/contextEntities/type/iot/id/urn:ngsi-ld:eduardo:001/attributes/pressure"),
    ("dataColetaDados", "http://20.62.95.153:8666/STH/v1/contextEntities/type/iot/id/urn:ngsi-ld:eduardo:001/attributes/dataColetaDados"),
    ("acelerometroEixoX", "http://20.62.95.153:8666/STH/v1/contextEntities/type/iot/id/urn:ngsi-ld:eduardo:001/attributes/acelerometroEixoX"),
    ("acelerometroEixoY", "http://20.62.95.153:8666/STH/v1/contextEntities/type/iot/id/urn:ngsi-ld:eduardo:001/attributes/acelerometroEixoY"),
    ("acelerometroEixoZ", "http://20.62.95.153:8666/STH/v1/contextEntities/type/iot/id/urn:ngsi-ld:eduardo:001/attributes/acelerometroEixoZ"),
    ("giroscopioPitch", "http://20.62.95.153:8666/STH/v1/contextEntities/type/iot/id/urn:ngsi-ld:eduardo:001/attributes/giroscopioPitch"),
    ("giroscopioYaw", "http://20.62.95.153:8666/STH/v1/contextEntities/type/iot/id/urn:ngsi-ld:eduardo:001/attributes/giroscopioYaw"),
    ("giroscopioRow", "http://20.62.95.153:8666/STH/v1/contextEntities/type/iot/id/urn:ngsi-ld:eduardo:001/attributes/giroscopioRow"),
    ("location", "http://20.62.95.153:8666/STH/v1/contextEntities/type/iot/id/urn:ngsi-ld:eduardo:001/attributes/location"),
    ("engineload", "http://20.62.95.153:8666/STH/v1/contextEntities/type/iot/id/urn:ngsi-ld:eduardo:001/attributes/engineload"),
    ("idCorrida", "http://20.62.95.153:8666/STH/v1/contextEntities/type/iot/id/urn:ngsi-ld:eduardo:001/attributes/idCorrida"),
    ("throttlePosition", "http://20.62.95.153:8666/STH/v1/contextEntities/type/iot/id/urn:ngsi-ld:eduardo:001/attributes/throttlePosition"),
]

# DataFrames para armazenar os resultados
dfs = {attribute: pd.DataFrame() for attribute, _ in attributes}

# Requisições paralelas para todos os atributos
with ThreadPoolExecutor() as executor:
    future_to_attribute = {
        executor.submit(fetch_paginated_data, url, attribute, 100, attribute == "location"): attribute
        for attribute, url in attributes
    }
    for future in future_to_attribute:
        attribute = future_to_attribute[future]
        try:
            result = future.result()
            dfs[attribute] = result
        except Exception as e:
            print(f"Erro ao buscar {attribute}: {e}")

# Combinar os DataFrames baseando-se nos timestamps
df = dfs["velocidade"]
for attribute in [attr for attr, _ in attributes if attr != "velocidade"]:
    df = df.merge(
        dfs[attribute],
        how="outer",
        left_on="recvTime_velocidade",
        right_on=f"recvTime_{attribute}"
    )

# Limpar o DataFrame final

df = df.drop(columns=['recvTime_velocidade', 'recvTime_rpm', 'recvTime_temperatura', 'recvTime_pressao', 'recvTime_dataColetaDados', 'recvTime_acelerometroEixoX', 'recvTime_acelerometroEixoY', 'recvTime_acelerometroEixoZ','recvTime_giroscopioPitch', 'recvTime_giroscopioYaw', 'recvTime_giroscopioRow', 'recvTime_location', 'recvTime_engineload', 'recvTime_idCorrida', 'recvTime_throttlePosition'])
df = df.dropna()

# Exibir o resultado
df_raw = df.copy()

df_raw



### Opcional - Forma de coleta a partir de arquivos textos

In [ ]:
import pandas as pd
import json
import requests

# URL do arquivo JSON
file_path = r"C:\Users\vidal\Downloads\TCC\ClassificacaoCondutores\DadosConsiderados\EricNormal.txt"

# Lista para armazenar os dados
data_list = []

# Ler o conteúdo do arquivo JSON linha por linha
with open(file_path, 'r') as file:
    for line in file:
        line = line.strip()
        if line:  # Verifica se a linha não está vazia
            try:
                data_dict = json.loads(line)
                data_list.append({
                    'velocidade': float(data_dict['velocidade']['value']),
                    'rpm': float(data_dict['rpm']['value']),
                    'pressao': float(data_dict['pressure']['value']),
                    'temperatura': float(data_dict['temperature']['value']),
                    'engineload': float(data_dict['engineload']['value']),
                    'throttlePosition': float(data_dict['throttlePosition']['value']),
                    'location': data_dict['location']['value']['coordinates'],
                    'acelerometroEixoX': float(data_dict['acelerometroEixoX']['value']),
                    'acelerometroEixoY': float(data_dict['acelerometroEixoY']['value']),
                    'acelerometroEixoZ': float(data_dict['acelerometroEixoZ']['value']),
                    'giroscopioRow': float(data_dict['giroscopioRow']['value']),
                    'giroscopioPitch': float(data_dict['giroscopioPitch']['value']),
                    'giroscopioYaw': float(data_dict['giroscopioYaw']['value']),
                    'dataColetaDados': data_dict['dataColetaDados']['value']
                })
            except json.JSONDecodeError:
                print(f"Erro ao decodificar JSON na linha: {line}")

# Criar um DataFrame a partir da lista de dados
df_raw = pd.DataFrame(data_list)

# Exibir o DataFrame
df_raw

## Tratamento de dados 

A partir do dataset coletado, é preciso fazer algumas correções. 

- Converter as colunas para cada tipo
- Retirar casos inconsistentes. Exemplo: data = 0, location[0,0], dados com -999
- Encontrar as corridas que existem nas entidades
- Separar o location em lat e log

In [ ]:
import datetime

df = df_raw

#Conversão do tipo de dado.
columns = {
    'idCorrida': int,
    'velocidade': int,
    'rpm': float,
    'temperatura': int,
    'pressao': int,
    'engineload': float,
    'throttlePosition': float,
    'giroscopioYaw': float,
    'giroscopioPitch': float,
    'giroscopioRow': float,
    'acelerometroEixoX': float,
    'acelerometroEixoY': float,
    'acelerometroEixoZ': float,
    'dataColetaDados': datetime,
    'location': str,
}

#Retirar casos em que o dia veio como 0. É uma inconsistencia geral
df = df[df['dataColetaDados'] != '0']


for column, dtype in columns.items():
    if dtype == datetime:
        df[column] = pd.to_datetime(df[column])
    else:
        df[column] = df[column].astype(dtype)

df = df.set_index('dataColetaDados')

# Remover valores nulos ou inconsistentes em 'location'
df = df[df['location'].notna() & df['location'].str.startswith('[')]

# Separar latitude e longitude usando split
df['latitude'] = df['location'].apply(lambda x: x.strip('[]').split(', ')[0]).astype(float)
df['longitude'] = df['location'].apply(lambda x: x.strip('[]').split(', ')[1]).astype(float)

#Inconsistência de longitude e latitude serem 0
df = df[(df['latitude'] != 0) & (df['longitude'] != 0)]

#Removendo valores extremos. Normalmente eles valem -999
df = df[(df['velocidade'] != -999) & (df['rpm'] != -999) & (df['pressao'] != -999) & (df['temperatura'] != -999) & (df['engineload'] != -999) & (df['throttlePosition'] != -999)]

#Considero um unico dataframe, pois o processo é mais linear
df_processado = df[['velocidade', 'rpm', 'temperatura', 'pressao', 'engineload', 'throttlePosition', 'location', 'latitude', 'longitude', 'idCorrida', 'giroscopioYaw', 'giroscopioPitch', \
                    'giroscopioRow', 'acelerometroEixoX', 'acelerometroEixoY', 'acelerometroEixoZ']].copy()


## Tratamento de Dados - Correção de coordenada
- Correção de latitude e longitude usando API google Snap to Roads.

In [ ]:
# Transformar o índice em uma coluna e resetar o índice
df_processado = df_processado.reset_index()

# Renomear a coluna do índice, se necessário
df_processado = df_processado.rename(columns={'index': 'novo_indice'})

# Chave para acesso a API do Google Maps. Substitua pela sua chave
api_key = 

# Número máximo de coordenadas por batch
batch_size = 100
new_coords = {}
new_place_ids = {}

# Iterar sobre o DataFrame em batches
for start in range(0, len(df_processado), batch_size):
    end = start + batch_size
    batch = df_processado.iloc[start:end]

    # Criar o path para a requisição
    path = '|'.join([f"{lat},{lon}" for lat, lon in zip(batch['latitude'], batch['longitude'])])

    # Fazer a requisição para o Google Maps API
    url = f"https://roads.googleapis.com/v1/snapToRoads?path={path}&key={api_key}"
    response = requests.get(url)
    snapped_points = response.json()

    # Extrair as novas coordenadas e placeIds, ajustando o índice para o DataFrame original
    for point in snapped_points.get('snappedPoints', []):
        relative_index = point['originalIndex']  # Índice dentro do batch (0-99)
        original_index = relative_index + start  # Índice absoluto no DataFrame original
        # Adicionar nova coordenada
        new_coords[original_index] = (
            point['location']['latitude'], 
            point['location']['longitude']
        )
        # Adicionar placeId, se disponível
        if 'placeId' in point:
            new_place_ids[original_index] = point['placeId']

# Adicionar as novas coordenadas ao DataFrame original
df_processado['new_latitude'] = df_processado.index.map(lambda idx: new_coords[idx][0] if idx in new_coords else None)
df_processado['new_longitude'] = df_processado.index.map(lambda idx: new_coords[idx][1] if idx in new_coords else None)

# Adicionar a coluna de placeId ao DataFrame original
df_processado['placeId'] = df_processado.index.map(lambda idx: new_place_ids[idx] if idx in new_place_ids else None)

# Exibir o DataFrame atualizado
df_processado

## Critérios - Limite de Velocidade

- Por conta do baixo limite de requisições gratuitas da API da Azure, é preciso reduzir as requisições. Os 3 blocos abaixo avaliam limites de velocidade já coletados, e só requisitão os novos casos

In [ ]:
#Linhas que não receberam novas coordenadas são removidas
df_processado = df_processado.dropna(subset=['new_latitude', 'new_longitude'])

#Parquet que armazena placeId e o limite de velocidade
df_limites_cache = pd.read_parquet("limites_velocidade.parquet")

#Verificando se existem novos limites de velocidade a ser capturado
df_limites_faltantes = df_processado.merge(df_limites_cache, on='placeId', how='left')
df_limites_faltantes = df_limites_faltantes[df_limites_faltantes['limite_velocidade'].isna()]

if(len(df_limites_faltantes) > 0):
    
    #Criar um novo parquet com os novos limites
    df_limites_faltantes = df_limites_faltantes.drop_duplicates(subset=['placeId'], ignore_index=True)
    print(len(df_limites_faltantes))
    df_limites_faltantes = df_limites_faltantes.drop(columns=['limite_velocidade', 'Street'])


- Utilizando API Azure para encontrar limite da via ( Somente os casos que ainda não tem limite)

In [ ]:
if(len(df_limites_faltantes) > 0):

    import json

    # Chave para acesso a API do Azure. Substitua pela sua chave
    key =
    
    # Definindo o URL do endpoint batch
    url = f"https://atlas.microsoft.com/search/address/reverse/batch/json?api-version=1.0&subscription-key={key}"

    # Criando o payload dinamicamente a partir do DataFrame. Não precisa ter limite de 100 coordenadas para a API da Azure
    batch_items = []

    #Precisa resetar o index mais uma vez
    # Transformar o índice em uma coluna e resetar o índice
    #df_limites_faltantes = df_limites_faltantes.reset_index()

    # Renomear a coluna do índice, se necessário
    #df_limites_faltantes = df_limites_faltantes.rename(columns={'index': 'novo_indice'})

    for _, row in df_limites_faltantes.iterrows():
        latitude = row['new_latitude']
        longitude = row['new_longitude']
        # Adiciona um item ao batch usando o formato específico da query
        batch_items.append({
            "query": f"?query={latitude},{longitude}&returnSpeedLimit=true"
        })

    # Monta o payload final
    payload = {
        "batchItems": batch_items
    }

    headers = {
        'Content-Type': 'application/json'  # Importante: definir corretamente o Content-Type
    }


    # Fazendo a requisição para criar o job
    response = requests.post(url, data=json.dumps(payload), headers=headers)

    # Verificando o resultado da criação do job
    if response.status_code == 202:
        # O job foi aceito e a URL para verificar o status foi retornada
        operation_location = response.headers['location']
        print(f"Job criado com sucesso! Verifique o status em: {operation_location}")

        result_response = requests.get(operation_location)
        result_data = result_response.json()

        # Lista para armazenar os dados extraídos
        data = []

        # Extraindo os dados do JSON
        for item in result_data['batchItems']:
            address_info = item.get('response', {}).get('addresses', [{}])[0].get('address', {})
            street_name = address_info.get('street', 'Desconhecido')
            speed_limit = address_info.get('speedLimit', 'Sem velocidade')
            data.append({'Street': street_name, 'Speed Limit': speed_limit})

        # Criando um DataFrame a partir dos dados extraídos
        df_limites = pd.DataFrame(data)

        # Combinando os DataFrames
        df_limites_faltantes = df_limites_faltantes.merge(df_limites, left_index=True, right_index=True)

    else:
        print(f"Erro ao criar o job: {response.status_code} - {response.text}")

- Depois de encontrar os limites faltantes, precisa salvar no parquet.
- Observação: podem haver casos que não encontrou um limite para o placeId. Mesmo nesse caso, o placeId é salvo com limite 0, para evitar redundancia e um gasto maior da API

In [ ]:
if(len(df_limites_faltantes) > 0):
    #Captura somente o valor numerico do limite de velocidade e converte para float
    df_limites_faltantes['limite_velocidade'] = df_limites_faltantes['Speed Limit'].str.extract(r'(\d+)').astype(float)

    #Adicionar limite de velocidade como 0 nos casos em que não foi possível encontrar
    df_limites_faltantes.loc[df_limites_faltantes['limite_velocidade'].isna(), 'limite_velocidade'] = 0

    #Passar os novos limites para o parquet - Os placeId serao unicos???
    df_limites_cache = pd.concat([df_limites_cache, df_limites_faltantes[['placeId', 'Street','limite_velocidade']]], ignore_index=True)
    df_limites_cache.to_parquet('limites_velocidade.parquet')

#Adicionar coluna de limite de velocidade
df_processado = df_processado.merge(df_limites_cache, on='placeId', how='left')

#Retira da analise os casos em que a velocidade é 0 ou o limite de velocidade é 0. Pois podem representar tanto uma situação de um veiculo parado, como uma inconsistencia na base de dados.
#df_processado = df_processado[(df_processado['velocidade'] > 0) & (df_processado['limite_velocidade'] != 0)]

    

## Critérios - Aceleração e Desaceleração

In [ ]:
df_processado = df_processado.set_index('dataColetaDados')

#Precisa converter a velocidade para m/s, pois o limite de velocidade vem em km/h
df_processado['velocidade_convertida'] = df_processado['velocidade'] / 3.6
df_processado['diff_tempo'] = df_processado.index.to_series().diff().dt.total_seconds()
df_processado['diff_velocidade'] = df_processado['velocidade_convertida'].diff()  

df_processado['aceleracao_derivada'] = df_processado['diff_velocidade'] / df_processado['diff_tempo']

#Inconsistencia de corridas que os dados deram muitos alto, ou muito baixo, uma situação irreal
df_processado = df_processado[(df_processado['aceleracao_derivada'] > -10) & (df_processado['aceleracao_derivada'] < 10)]

## Gráficos - Visulização dos Critérios

In [ ]:
#Bibliotecas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
#Lendo tabela que já foi processada - OPCIONAL
df_processado_normal = pd.read_parquet(r"C:\Users\vidal\Downloads\normal.parquet")
df_processado_normal = df_processado_normal[df_processado_normal['limite_velocidade'] != 0]

df_processado = pd.read_parquet(r"C:\Users\vidal\Downloads\agressivo.parquet")
df_processado = df_processado[df_processado['limite_velocidade'] != 0]

- Velocidade x Limite de velocidade

In [ ]:
df_processado_grafico = df_processado.copy()

# Criar máscara para áreas abaixo e acima do limite de velocidade
abaixo_limite = df_processado_grafico['velocidade'] <= df_processado_grafico['limite_velocidade']
acima_limite = df_processado_grafico['velocidade'] > df_processado_grafico['limite_velocidade']

# Plotar o gráfico
plt.figure(figsize=(14, 8))

# Plotar a velocidade ao longo do tempo
plt.plot(df_processado_grafico.index, df_processado_grafico['velocidade'], label='Velocidade', linewidth=2, color='darkblue')
plt.plot(df_processado_grafico.index, df_processado_grafico['limite_velocidade'], label='Limite de Velocidade', linewidth=2, color='gray', linestyle='--')

# Preencher áreas abaixo do limite (azul claro)
plt.fill_between(
    df_processado_grafico.index,
    df_processado_grafico['velocidade'],
    df_processado_grafico['limite_velocidade'],
    where=abaixo_limite,
    interpolate=True,
    color='blue',
    alpha=0.2,
    label='Abaixo do Limite'
)

# Preencher áreas acima do limite (vermelho claro)
plt.fill_between(
    df_processado_grafico.index,
    df_processado_grafico['velocidade'],
    df_processado_grafico['limite_velocidade'],
    where=acima_limite,
    interpolate=True,
    color='red',
    alpha=0.2,
    label='Acima do Limite'
)

# Estilo do gráfico
plt.title('Critério Velocidade x Limite de Velocidade', fontsize=16, fontweight='bold')
plt.xlabel('Tempo', fontsize=12)
plt.ylabel('Velocidade (km/h)', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.legend(fontsize=10, loc='upper left')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()

# Mostrar o gráfico
plt.show()

- Aceleração e Desaceleração 

In [ ]:

# Filtrar apenas as corridas que tenham pelo menos 10 minutos de duração
df_selecionadas = df_processado.groupby('idCorrida').filter(lambda x: (x.index[-1] - x.index[0]).total_seconds() >= 600)

# Parâmetros
amostras_por_bloco = 500  # Número de pontos uniformes para reamostragem

# Função para reamostrar os dados para um número fixo de pontos
def reamostrar_bloco(bloco_df, n_pontos):
    tempo_normalizado = np.linspace(0, 1, len(bloco_df))  # Normalizar o tempo
    indices_interpolados = np.linspace(0, 1, n_pontos)    # Pontos interpolados uniformes
    valores_interpolados = np.interp(indices_interpolados, tempo_normalizado, bloco_df['aceleracao_derivada'].values)
    return valores_interpolados

# Preparar os dados alinhados
dados_grafico = []

# Encontrar o tempo total das corridas selecionadas
tempo_maximo = 0

for corrida_id, corrida_df in df_selecionadas.groupby('idCorrida'):
    corrida_df = corrida_df.sort_index()
    inicio_corrida = corrida_df.index[0]
    fim_corrida = corrida_df.index[-1]
    
    duracao_corrida = (fim_corrida - inicio_corrida).total_seconds() / 60  # Converter para minutos
    tempo_maximo = max(tempo_maximo, duracao_corrida)

    # Reamostrar toda a corrida em um único bloco
    bloco_reamostrado = reamostrar_bloco(corrida_df, amostras_por_bloco)
    dados_grafico.append(bloco_reamostrado)

# Converter os dados em uma matriz
dados_grafico_array = np.array(dados_grafico)

# Calcular a mediana (antes era a média)
mediana = np.median(dados_grafico_array, axis=0)

# Gerar eixo X ajustado ao tempo total da corrida
tempo_em_minutos = np.linspace(0, tempo_maximo, amostras_por_bloco)

# Criar o gráfico atualizado sem percentis, mantendo apenas a mediana
plt.figure(figsize=(14, 7))
plt.plot(tempo_em_minutos, mediana, label='Mediana', color='darkblue', linewidth=2)

# Adicionar linhas tracejadas para aceleração agressiva e desaceleração agressiva
plt.axhline(2.0, color='red', linestyle='--', linewidth=1.5, label='Aceleração Agressiva (2.0 m/s²)')
plt.axhline(-3.5, color='red', linestyle='--', linewidth=1.5, label='Desaceleração Agressiva (-3.5 m/s²)')

# Personalizar o gráfico
plt.title('Padrões de Aceleração Derivada', fontsize=16, fontweight='bold')
plt.xlabel('Tempo (minutos)', fontsize=12)
plt.ylabel('Aceleração Derivada', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.axhline(0, color='gray', linestyle='--', linewidth=1)  # Linha de referência
plt.legend(fontsize=10)
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

## Classificação - Segmentação

- Utilizando uma lógica semelhante ao artigo. Calcula quantas vezes um critério de gravidade acontece em X metros, tanto para uma pessoa comum, quanto para uma pessoa agressiva. A partir disso classifica.

Consultar: https://onlinelibrary.wiley.com/doi/10.1155/2018/9530470. Tem uma ampla gama de critérios

O bloco abaixo busca calcular distancia entre lat e log, utilizando uma função chamada Haversine.

In [ ]:
import numpy as np

# Cópia do dataframe (ajuste conforme o necessário)
df_classificacao_eventos = df_processado[['velocidade', 'rpm', 'temperatura', 'pressao', 'engineload', 'throttlePosition', 'limite_velocidade', 'aceleracao_derivada', \
                                           'new_latitude', 'new_longitude', 'Street', 'idCorrida', 'giroscopioYaw', 'giroscopioPitch', 'giroscopioRow', 'acelerometroEixoX', \
                                            'acelerometroEixoY', 'acelerometroEixoZ' ]].copy()


# Função de Haversine - calcula a distância entre dois pontos geográficos
def haversine(lat1, lon1, lat2, lon2):
    # Conversão de graus para radianos
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Diferenças
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Fórmula de Haversine
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    # Raio médio da Terra (em km)
    R = 6371.0
    return R * c  # Distância em quilômetros

# Adiciona uma coluna de distância entre pontos consecutivos
df_classificacao_eventos["distance"] = np.where(
    df_classificacao_eventos["idCorrida"] != df_classificacao_eventos["idCorrida"].shift(), 
    0,  # Caso a corrida mude, a distância é 0
    haversine(
        df_classificacao_eventos["new_latitude"].shift(), 
        df_classificacao_eventos["new_longitude"].shift(), 
        df_classificacao_eventos["new_latitude"], 
        df_classificacao_eventos["new_longitude"]
    )
)

# Preenche valores NaN com 0 (para a primeira linha)
df_classificacao_eventos["distance"].fillna(0, inplace=True)

# Calcula a distância total por corrida (opcional)
distancias_por_corrida = df_classificacao_eventos.groupby("idCorrida")["distance"].sum()

# Calcula a distância total
total_distance = df_classificacao_eventos["distance"].sum()

print(f"Distância total percorrida: {total_distance:.2f} km")


- Gera a quantidade de vezes que ocorreu um evento. No fim a classificação é geral, levando em consideração a quantidade total de km percorridos

In [ ]:
#Objetivo é calcular a quantidade de vezes que um critério ocorre.
#Critérios:
#ALV1 -> Até 20% acima do limite de velocidade. Existe uma tolerencia de 7 km quando limite é de até 100km/h. Após isso, a tolerancia é de 7%
#ALV2 -> 20% acima, até 50% acima do limite de velocidade
#ALV3 -> Mais de 50% acima do limite de velocidade
#AA -> Aceleração acima de 3 m/s²
#DA -> Desaceleração acima de 4 m/s²

#df_classificacao_eventos = df_classificacao_eventos[df_classificacao_eventos['limite_velocidade'] != 0]

# Calcular os critérios sem utilizar iterrows
df_classificacao_eventos['tolerancia'] = np.where(
    df_classificacao_eventos['limite_velocidade'] <= 100,
    7,
    df_classificacao_eventos['limite_velocidade'] * 0.07
)

# Critérios de classificação considerando a tolerância
df_classificacao_eventos['ALV1'] = (
    (df_classificacao_eventos['velocidade'] > (df_classificacao_eventos['limite_velocidade'] + df_classificacao_eventos['tolerancia'])) &
    (df_classificacao_eventos['velocidade'] <= (df_classificacao_eventos['limite_velocidade'] * 1.2 + df_classificacao_eventos['tolerancia']))
).astype(int)

df_classificacao_eventos['ALV2'] = (
    (df_classificacao_eventos['velocidade'] > (df_classificacao_eventos['limite_velocidade'] * 1.2 + df_classificacao_eventos['tolerancia'])) &
    (df_classificacao_eventos['velocidade'] <= (df_classificacao_eventos['limite_velocidade'] * 1.5 + df_classificacao_eventos['tolerancia']))
).astype(int)

df_classificacao_eventos['ALV3'] = (
    df_classificacao_eventos['velocidade'] > (df_classificacao_eventos['limite_velocidade'] * 1.5 + df_classificacao_eventos['tolerancia'])
).astype(int)

df_classificacao_eventos['AA'] = (df_classificacao_eventos['aceleracao_derivada'] >= 2.0).astype(int)

df_classificacao_eventos['DA'] = (df_classificacao_eventos['aceleracao_derivada'] <= -3.5).astype(int)


print(f'Dentro de {total_distance:.2f} km, ocorreram as seguintes quantidades de eventos:')
df_eventos = df_classificacao_eventos[['ALV1', 'ALV2', 'ALV3', 'AA', 'DA']].sum().to_frame().transpose()
df_eventos


- A partir da classificação geral, busca quebrar em trechos de X metros, existe um problema de trechos maiores do que X metros, ou seja, gera menos trechos do que a quantidade total percorrida. Exemplo: Uma base com uma distancia de 50 km, pode ter somente 43 trechos. No fim os trechos não tem necessariamente a distancia esperada

In [ ]:
df_classificacao_eventos['distancia_acumulada'] = df_classificacao_eventos['distance'].cumsum()
df_classificacao_eventos['segmento'] = (df_classificacao_eventos['distancia_acumulada'] // 0.1).astype(int) #100 metros

df_classificacao_eventos['latitude_inicial'] = df_classificacao_eventos.groupby('segmento')['new_latitude'].transform('first')
df_classificacao_eventos['longitude_inicial'] = df_classificacao_eventos.groupby('segmento')['new_longitude'].transform('first')
df_classificacao_eventos['latitude_final'] = df_classificacao_eventos.groupby('segmento')['new_latitude'].transform('last')
df_classificacao_eventos['longitude_final'] = df_classificacao_eventos.groupby('segmento')['new_longitude'].transform('last')
df_classificacao_eventos['soma_evento_segmento_aa'] = df_classificacao_eventos.groupby('segmento')['AA'].transform('sum')
df_classificacao_eventos['soma_evento_segmento_da'] = df_classificacao_eventos.groupby('segmento')['DA'].transform('sum')
df_classificacao_eventos['soma_evento_segmento_alv1'] = df_classificacao_eventos.groupby('segmento')['ALV1'].transform('sum')
df_classificacao_eventos['soma_evento_segmento_alv2'] = df_classificacao_eventos.groupby('segmento')['ALV2'].transform('sum')
df_classificacao_eventos['soma_evento_segmento_alv3'] = df_classificacao_eventos.groupby('segmento')['ALV3'].transform('sum')


#Esse df é só para visualizar os casos
df_segmentos = df_classificacao_eventos.groupby('segmento').agg({'AA': 'sum', 'DA': 'sum', \
                                                                    'ALV1': 'sum', 'ALV2': 'sum',
                                                                      'ALV3':  'sum', 'latitude_inicial': 'first', \
                                                                'longitude_inicial': 'first', 'latitude_final' : 'first', 
                                                                'longitude_final': 'first'  }).reset_index()



df_segmentos


- Faz a classificação de cada segmento. Considerando somente dois grupos: Normal e Agressivo.

In [ ]:
df_classificacao_eventos['total_eventos'] = df_classificacao_eventos['soma_evento_segmento_aa'] + df_classificacao_eventos['soma_evento_segmento_da'] + \
    df_classificacao_eventos['soma_evento_segmento_alv1'] + df_classificacao_eventos['soma_evento_segmento_alv2'] + df_classificacao_eventos['soma_evento_segmento_alv3']

df_classificacao_eventos.loc[df_classificacao_eventos['total_eventos'] >= 1, 'classificacao'] = 'AGRESSIVO'
df_classificacao_eventos.loc[df_classificacao_eventos['total_eventos'] == 0, 'classificacao'] = 'NORMAL'

df_classificacao_eventos[['segmento','ALV1', 'ALV2', 'ALV3', 'AA', 'DA', 'classificacao']].groupby('segmento').agg({'ALV1': 'sum', 'ALV2': 'sum', 'ALV3': 'sum', 'AA': 'sum', 'DA': 'sum', 'classificacao': 'first'}).head(15)


- Gera diferentes formas de classificação. Sempre considera a classificação levando como base o segmento de 100 metros, mas com ideias de classificação diferentes. 

In [ ]:
#Nesse bloco são feitos diferentes tipos de classificações.
# 1 - Classificação por segmento mantendo linha a linha. 
# 2 - Classificação por segmento, mas mantendo somente um registro por segmento
# 3 - Classificação considerando a moda do que mais acontece para o segmento. Mantem somente um registro por segmento

df_classificacao_eventos = df_classificacao_eventos.reset_index()

df_classificacao_eventos['motorista'] = 'eduardo'

df_final = pd.read_excel("df_final.xlsx")
df_final = pd.concat([df_final, df_classificacao_eventos], ignore_index=True)
df_final.to_excel("df_final.xlsx", index=False)



#df_classificacao_eventos_segmentos = df_classificacao_eventos.copy()
#df_classificacao_eventos_segmentos = df_classificacao_eventos_segmentos.groupby('segmento').agg({'velocidade': 'mean', 'rpm': 'mean', 'temperatura': 'mean', 'pressao': 'mean', 'engineload': 'mean', 'throttlePosition': 'mean', 'limite_velocidade': 'mean', 'aceleracao_derivada': 'mean', \
#                                           'new_latitude': 'first', 'new_longitude': 'first', 'Street': 'first', 'giroscopioYaw': 'mean', 'giroscopioPitch': 'mean', 'giroscopioRow': 'mean', 'acelerometroEixoX': 'mean', \
#                                           'acelerometroEixoY': 'mean', 'acelerometroEixoZ': 'mean', 'ALV1': 'sum', 'ALV2': 'sum', 'ALV3': 'sum', 'AA': 'sum', 'DA': 'sum', 'classificacao': 'first', 'motorista': 'first'}).reset_index()
#df_classificacao_eventos_segmentos = df_classificacao_eventos_segmentos.drop(columns=['segmento'])


#df_final_seg = pd.read_excel("df_ml_segmentos_validacao.xlsx")
#df_final_seg = pd.concat([df_final_seg, df_classificacao_eventos_segmentos], ignore_index=True)
#df_final_seg.to_excel("df_ml_segmentos_validacao.xlsx", index=False)

#df_classificacao_moda['total_eventos'] = df_classificacao_moda['ALV1'] + df_classificacao_moda['ALV2'] + df_classificacao_moda['ALV3'] + df_classificacao_moda['AA'] + df_classificacao_moda['DA']
#df_classificacao_moda.loc[df_classificacao_moda['total_eventos'] >= 1, 'classificacao'] = 'AGRESSIVO'
#df_classificacao_moda.loc[df_classificacao_moda['total_eventos'] == 0, 'classificacao'] = 'NORMAL'

#df_classificacao_moda= df_classificacao_moda.groupby('segmento').agg({
#    'velocidade': 'mean',
#    'rpm': 'mean',
#    'temperatura': 'mean',
#    'pressao': 'mean',
#    'engineload': 'mean',
#    'throttlePosition': 'mean',
#    'limite_velocidade': 'mean',
#    'aceleracao_derivada': 'mean',
#    'giroscopioYaw': 'mean',
#    'giroscopioPitch': 'mean',
#    'giroscopioRow': 'mean',
#    'acelerometroEixoX': 'mean',
#    'acelerometroEixoY': 'mean',
#    'acelerometroEixoZ': 'mean',
#    'ALV1': 'sum',
#    'ALV2': 'sum',
#    'ALV3': 'sum',
#    'AA': 'sum',
#    'DA': 'sum',
#    'classificacao': lambda x: x.mode()[0],
#    'motorista': 'first'
#}).reset_index()

#df_classificacao_moda = df_classificacao_moda.drop(columns=['segmento'])

#df_final_moda = pd.read_excel("df_ml_moda_validacao.xlsx")
#df_final_moda = pd.concat([df_final_moda, df_classificacao_moda], ignore_index=True)
#f_final_moda.to_excel("df_ml_moda_validacao.xlsx", index=False)



## Machine Learning - Validação

- Base Original

In [ ]:
import pandas as pd
df_ml = pd.read_excel(r"C:\Users\vidal\Downloads\TCC\ClassificacaoCondutores\df_final.xlsx")

df_ml = df_ml[[ 'velocidade', 'rpm', 'temperatura', 'pressao', 'engineload', 'throttlePosition', 'limite_velocidade', 'aceleracao_derivada',  'classificacao']]

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

scaler = StandardScaler()

x_scaled = scaler.fit_transform(df_ml.drop(columns=['classificacao']))

df_ml_scaled = pd.DataFrame(x_scaled, columns=df_ml.drop(columns=['classificacao']).columns)

df_ml_scaled['classificacao'] = df_ml['classificacao']

df_ml_scaled.head()


- Teste de correlação entre variaveis

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

le = LabelEncoder()
df_ml_scaled['classificacao'] = le.fit_transform(df_ml_scaled['classificacao'])

# Calcular a matriz de correlação
correlation_matrix = df_ml_scaled.corr()

# Configurar o tamanho da figura
plt.figure(figsize=(12, 10))

# Criar um mapa de calor da matriz de correlação
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)

# Configurar o título do gráfico
plt.title('Matriz de Correlação das Variáveis', fontsize=16)

# Mostrar o gráfico
plt.show()

#### Random Forest

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Separar as features e o target
X_train = df_ml_scaled.drop(columns=['classificacao'])
y_train = df_ml_scaled['classificacao']

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=42, stratify=y_train)

# Construir e treinar o modelo Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Fazer previsões e avaliar o modelo
y_pred_random = rf_model.predict(X_test)
print("Acurácia:", accuracy_score(y_test, y_pred_random))
print(classification_report(y_test, y_pred_random, target_names=le.classes_))

#### Regressão Logistica

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Separar as features e o target
X_train = df_ml_scaled.drop(columns=['classificacao'])
y_train = df_ml_scaled['classificacao']

# Dividir o conjunto de dados para treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=42, stratify=y_train)

# Construir e treinar o modelo de Regressão Logística
rl_model = LogisticRegression(random_state=42)
rl_model.fit(X_train, y_train)

# Fazer previsões e avaliar o modelo
y_pred_rl = rl_model.predict(X_test)
print("Acurácia:", accuracy_score(y_test, y_pred_rl))
print(classification_report(y_test, y_pred_rl, target_names=le.classes_))

#### SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Criar e treinar o modelo SVM
svm_model = SVC(kernel="rbf", C=10.0, decision_function_shape='ovo', class_weight='balanced')  # Testar 'linear' ou 'rbf'
svm_model.fit(X_train, y_train)

# Fazer previsões
y_pred_svm = svm_model.predict(X_test)
print("Acurácia:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm, target_names=le.classes_))


### Resultados - Visualização

In [ ]:
#Validação - Matriz de convulsão

from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


cf_matrix_rf = confusion_matrix(y_test, y_pred_random)
cf_matrix_rl = confusion_matrix(y_test, y_pred_rl)
cf_matrix_svm = confusion_matrix(y_test, y_pred_svm)


plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
sns.heatmap(cf_matrix_rf, annot=True, cmap='Blues', fmt='d', cbar=False)
plt.title('Matriz de Confusão - Random Forest')
plt.xlabel('Previsto')
plt.ylabel('Real')

plt.subplot(1, 3, 2)
sns.heatmap(cf_matrix_rl, annot=True, cmap='Blues', fmt='d', cbar=False)
plt.title('Matriz de Confusão - Regressão Logística')
plt.xlabel('Previsto')
plt.ylabel('Real')

plt.subplot(1, 3, 3)
sns.heatmap(cf_matrix_svm, annot=True, cmap='Blues', fmt='d', cbar=False)
plt.title('Matriz de Confusão - SVM')
plt.xlabel('Previsto')
plt.ylabel('Real')

## Mapa com o circuito classificado
- Gerando um mapa com os segmentos no folium

In [ ]:
df_processado_agressivo = pd.read_excel(r"C:\Users\vidal\Downloads\TCC\ClassificacaoCondutores\df_final.xlsx")

df_processado_agressivo = df_processado_agressivo[(df_processado_agressivo['motorista'] ==  'celso') & (df_processado_agressivo['idCorrida'] == 2)]

In [ ]:

df_processado_agressivo = df_processado_agressivo[df_processado_agressivo['limite_velocidade'] != 0]

# Seleciona as colunas usadas no treinamento
feature_cols = ['velocidade', 'rpm', 'temperatura', 'pressao', 
                'engineload', 'throttlePosition', 'limite_velocidade', 'aceleracao_derivada']

# Prever a classificação para o dataset "agressivo"
df_agressivo_ml = df_processado_agressivo.copy()
X_agressivo = df_agressivo_ml[feature_cols]
X_agressivo_scaled = scaler.transform(X_agressivo)
df_agressivo_ml['classificacao'] = svm_model.predict(X_agressivo_scaled)

# Combina os datasets classificados (mantendo as coordenadas) para uso no mapa
df_geracao = df_agressivo_ml

In [ ]:
import folium

# Cria o mapa centralizado na primeira coordenada
m = folium.Map(location=[df_geracao['new_latitude'].iloc[0], 
                         df_geracao['new_longitude'].iloc[0]], 
               zoom_start=15)

# Adiciona os pontos e linhas ao mapa
for i in range(len(df_geracao) - 1):
    # Coordenadas do ponto atual e do próximo ponto
    lat1, lon1 = df_geracao.iloc[i][['new_latitude', 'new_longitude']]
    lat2, lon2 = df_geracao.iloc[i + 1][['new_latitude', 'new_longitude']]
    
    # Classificação do ponto atual
    classificacao = df_geracao.iloc[i]['classificacao']
    
    # Define a cor da linha com base na classificação
    line_color = 'red' if classificacao == 0 else 'green'
    
    # Adiciona uma linha entre o ponto atual e o próximo
    folium.PolyLine([(lat1, lon1), (lat2, lon2)], color=line_color, weight=5).add_to(m)
    
    # Adiciona um marcador no ponto atual
    #folium.Marker(
    #    location=[lat1, lon1],
    #    popup=f"Coordenadas: ({lat1}, {lon1})<br>Classificação: {classificacao}",
    #   icon=folium.Icon(color='blue', icon='info-sign')
    #).add_to(m)

# Salva o mapa em um arquivo HTML e exibe o mapa
output_file = r"C:\Users\vidal\Downloads\TCC\ClassificacaoCondutores\percurso.html"
m.save(output_file)
output_file